# Worksheet 08

Name:  Emmalee Gomez  
UID:  U76971767

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.849404067306161, 3.687708930766738, 5.3297781745427555, 3.366121086294348, 5.813868969784585, 3.7162952352755854, 7.269835964330703, 4.498135691786759, 4.425819785703776, 5.925150101013855]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
mean = 8
stdev = 1

c2 = np.random.normal(mean, stdev , 10).tolist()
print(c2)

[9.20916244016338, 7.5808896240989885, 8.069606160122923, 6.296627548447907, 7.991082489187591, 7.822615714494064, 8.76685854856919, 7.542397226527863, 8.791034712942688, 9.428992704290561]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[9.428992704290561, 5.925150101013855, 4.425819785703776, 8.791034712942688, 7.542397226527863, 4.498135691786759, 7.269835964330703, 8.76685854856919, 3.7162952352755854, 5.813868969784585]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean and variance.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.925150101013855, 4.425819785703776, 4.498135691786759, 3.7162952352755854, 5.813868969784585]
[9.428992704290561, 8.791034712942688, 7.542397226527863, 7.269835964330703, 8.76685854856919]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 4.875853956712913,  mean_2 = 8.359823831332202
var_1 = 0.7341345668777498,  var_2 = 12.808246481016932


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  9.428992704290561
probability of observing that point if it came from cluster 0 =  2.4125462444828393e-09
probability of observing that point if it came from cluster 1 =  0.031038969166103395
point =  5.925150101013855
probability of observing that point if it came from cluster 0 =  0.19567108104617226
probability of observing that point if it came from cluster 1 =  0.030589631212765415
point =  4.425819785703776
probability of observing that point if it came from cluster 0 =  0.4503329036779042
probability of observing that point if it came from cluster 1 =  0.029712211901316177
point =  8.791034712942688
probability of observing that point if it came from cluster 0 =  3.62363224402674e-07
probability of observing that point if it came from cluster 1 =  0.03112965195327786
point =  7.542397226527863
probability of observing that point if it came from cluster 0 =  0.0007418144096563252
probability of observing that point if it came from cluster 1 =  0.031083931333095297
point 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[0])**2, c2)) / len(c2)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.4576323608102674,  P(C_2) = 0.5423676391897325
mean_1 = 4.927572296956368,  mean_2 = 8.044031456635132
var_1 = 0.7368093535952877,  var_2 = 12.450550988987455


The values are fairly close.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [12]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  9.428992704290561
probability of observing that point if it came from cluster 0 =  4.253418125618645e-09
probability of observing that point if it came from cluster 1 =  0.031844511341697884
point =  5.925150101013855
probability of observing that point if it came from cluster 0 =  0.21652345689720678
probability of observing that point if it came from cluster 1 =  0.03158147184789056
point =  4.425819785703776
probability of observing that point if it came from cluster 0 =  0.42939442110212367
probability of observing that point if it came from cluster 1 =  0.030717292947388283
point =  8.791034712942688
probability of observing that point if it came from cluster 0 =  5.797453675409769e-07
probability of observing that point if it came from cluster 1 =  0.03198451921853348
point =  7.542397226527863
probability of observing that point if it came from cluster 0 =  0.0009970834963320561
probability of observing that point if it came from cluster 1 =  0.032016142242842396
point 

The probabilities for both seem wrong, but they do both favor cluster 1 or 2 for each point.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [18]:
clusters = []

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    label = 0 if p[1] > p[2] else 1
    clusters.append(label)
    
    print(label)

9.428992704290561
1
5.925150101013855
0
4.425819785703776
0
8.791034712942688
1
7.542397226527863
1
4.498135691786759
0
7.269835964330703
1
8.76685854856919
1
3.7162952352755854
0
5.813868969784585
0


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      agree       |
| A  C |      disagree      |
| A  D |      disagree       |
| A  E |      disagree       |
| B  C |      agree       |
| B  D |      disagree       |
| B  E |      disagree       |
| C  D |      disagree       |
| C  E |      disagree      |
| D  E |      agree     |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

n choose 2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

- Cluster 1 has 1 agreements
- Cluster 2 has 6 agreements
- There would be 10 total agreements
- Between C and P, 21 disagreements

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

Run a for loop from 0 to num points length. Then run a for loop from number of the previous loop to points length. Then compare each and see if they agree or disagree.